<a href="https://colab.research.google.com/github/mprudd2/Project_Two_Team_Hodl/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/mprudd2/Project_Two_Team_Hodl.git

fatal: destination path 'Project_Two_Team_Hodl' already exists and is not an empty directory.


In [462]:
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import DateOffset
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
import numpy as np


In [463]:
btc_df = pd.read_csv(
    Path("/content/Project_Two_Team_Hodl/Resources/Bitstamp_BTCUSD_d.csv"),
    index_col="date",
    infer_datetime_format=True,
    parse_dates=True
)



In [464]:
btc_df.head()

,symbol,open,high,low,close,Volume BTC,Volume USD
date,,,,,,,
2022-03-17,BTC/USD,41142.76,41187.60,40961.35,41049.85,8.817627,3.619623e+05
2022-03-16,BTC/USD,39302.80,41693.97,38865.67,41142.76,5987.103889,2.463260e+08
2022-03-15,BTC/USD,39688.55,39896.13,38142.35,39302.70,1638.684030,6.440471e+07
2022-03-14,BTC/USD,37763.37,39913.42,37567.19,39665.71,1439.767682,5.710941e+07
2022-03-13,BTC/USD,38798.68,39312.50,37595.62,37790.54,737.880132,2.788489e+07


In [465]:
btc_df["actual returns"] = btc_df["close"].pct_change()
btc_df.head()

,symbol,open,high,low,close,Volume BTC,Volume USD,actual returns
date,,,,,,,,
2022-03-17,BTC/USD,41142.76,41187.60,40961.35,41049.85,8.817627,3.619623e+05,NaN
2022-03-16,BTC/USD,39302.80,41693.97,38865.67,41142.76,5987.103889,2.463260e+08,0.002263
2022-03-15,BTC/USD,39688.55,39896.13,38142.35,39302.70,1638.684030,6.440471e+07,-0.044724
2022-03-14,BTC/USD,37763.37,39913.42,37567.19,39665.71,1439.767682,5.710941e+07,0.009236
2022-03-13,BTC/USD,38798.68,39312.50,37595.62,37790.54,737.880132,2.788489e+07,-0.047274


In [466]:
btc_df = btc_df.dropna()
display(btc_df.head())
display(btc_df.tail())

,symbol,open,high,low,close,Volume BTC,Volume USD,actual returns
date,,,,,,,,
2022-03-16,BTC/USD,39302.80,41693.97,38865.67,41142.76,5987.103889,2.463260e+08,0.002263
2022-03-15,BTC/USD,39688.55,39896.13,38142.35,39302.70,1638.684030,6.440471e+07,-0.044724
2022-03-14,BTC/USD,37763.37,39913.42,37567.19,39665.71,1439.767682,5.710941e+07,0.009236
2022-03-13,BTC/USD,38798.68,39312.50,37595.62,37790.54,737.880132,2.788489e+07,-0.047274
2022-03-12,BTC/USD,38729.79,39458.58,38653.89,38822.02,962.855666,3.738000e+07,0.027295


,symbol,open,high,low,close,Volume BTC,Volume USD,actual returns
date,,,,,,,,
2014-12-02,BTC/USD,378.39,382.86,375.23,379.25,2593576.46,6832.53,0.006849
2014-12-01,BTC/USD,376.40,382.31,373.03,378.39,2520662.37,6660.56,-0.002268
2014-11-30,BTC/USD,376.57,381.99,373.32,373.34,1145566.61,3046.33,-0.013346
2014-11-29,BTC/USD,376.42,386.60,372.25,376.72,2746157.05,7245.19,0.009053
2014-11-28,BTC/USD,363.59,381.34,360.57,376.28,3220878.18,8617.15,-0.001168


In [467]:
short_window = 25

btc_df['sma_fast'] = btc_df['close'].rolling(window = short_window).mean()

In [468]:
long_window = 100

btc_df['sma_slow'] = btc_df['close'].rolling(window = long_window).mean()

In [469]:
btc_df = btc_df.dropna()

In [470]:
X = btc_df[['sma_fast', 'sma_slow']].shift().dropna().copy()


In [471]:
X.head()

,sma_fast,sma_slow
date,,
2021-12-06,48495.2956,42697.3512
2021-12-05,48668.9208,42791.4736
2021-12-04,48761.4212,42893.0788
2021-12-03,48871.4856,42988.7714
2021-12-02,49119.5012,43148.3029


In [472]:
X.tail()

,sma_fast,sma_slow
date,,
2014-12-02,343.0392,274.2128
2014-12-01,345.5736,275.0497
2014-11-30,347.4740,275.8825
2014-11-29,349.6320,276.7135
2014-11-28,351.8152,277.6107


In [473]:
btc_df['signal'] = 0.0

In [474]:
btc_df.loc[(btc_df['actual returns'] >= 0), 'signal'] = 1


btc_df.loc[(btc_df['actual returns'] < 0), 'signal'] = -1

In [475]:
y = btc_df['signal'].copy()
y.value_counts()

-1.0    1381
 1.0    1186
Name: signal, dtype: int64

In [476]:

    
y.drop(index = y.index[0], 
        axis=0, 
        inplace=True)
y.head()

date
2021-12-06   -1.0
2021-12-05   -1.0
2021-12-04   -1.0
2021-12-03    1.0
2021-12-02    1.0
Name: signal, dtype: float64

In [477]:

training_begin = X.index.min()


print(training_begin)




2014-11-28 00:00:00


In [478]:
training_end = X.index.min() + DateOffset(months=60)


print(training_end)

2019-11-28 00:00:00


In [479]:

X_train = X
y_train = y



In [480]:
y_train.head()

date
2021-12-06   -1.0
2021-12-05   -1.0
2021-12-04   -1.0
2021-12-03    1.0
2021-12-02    1.0
Name: signal, dtype: float64

In [481]:
X_train.head()

,sma_fast,sma_slow
date,,
2021-12-06,48495.2956,42697.3512
2021-12-05,48668.9208,42791.4736
2021-12-04,48761.4212,42893.0788
2021-12-03,48871.4856,42988.7714
2021-12-02,49119.5012,43148.3029


In [482]:
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

X_test.tail()

,sma_fast,sma_slow
date,,
2014-12-02,343.0392,274.2128
2014-12-01,345.5736,275.0497
2014-11-30,347.4740,275.8825
2014-11-29,349.6320,276.7135
2014-11-28,351.8152,277.6107


In [483]:
from sklearn.preprocessing import StandardScaler
from sklearn import svm


In [484]:
scaler = StandardScaler()

In [485]:
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [486]:
svm_model = svm.SVC()

svm_model = svm_model.fit(X_train_scaled, y_train)

training_signal_predictions = svm_model.predict(X_train_scaled)

training_signal_predictions[:10]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [487]:
from sklearn.metrics import classification_report

training_report = classification_report(y_train, training_signal_predictions)
print(training_report)

              precision    recall  f1-score   support

        -1.0       0.54      1.00      0.70      1381
         1.0       0.00      0.00      0.00      1185

    accuracy                           0.54      2566
   macro avg       0.27      0.50      0.35      2566
weighted avg       0.29      0.54      0.38      2566



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [488]:

testing_signal_predictions = svm_model.predict(X_test_scaled)

In [489]:
testing_report = classification_report(y_test, testing_signal_predictions)
print(testing_report)

              precision    recall  f1-score   support

        -1.0       0.54      1.00      0.70       982
         1.0       0.00      0.00      0.00       845

    accuracy                           0.54      1827
   macro avg       0.27      0.50      0.35      1827
weighted avg       0.29      0.54      0.38      1827



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
